# Load facts from a file.

## Parse a single fact line into a key-value pair

In [47]:
def parse_fact_line(line):
    line = line.strip()
    if not line or line.startswith('#'):
        return None

    if ' is ' in line:
        key, value = line.split(' is ')
        return key.strip(), value.strip()
    elif '=' in line:
        key, value = line.split('=')
        key = key.strip()
        try:
            value = float(value.strip())
        except ValueError:
            value = value.strip()
        return key, value
    else:
        return line.strip(), True

In [48]:
def load_facts(filename):
    facts = {}
    goal = None

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue

            if line.startswith("#goal prove"):
                goal = line[12:].strip()
                continue

            result = parse_fact_line(line)
            if result:
                key, value = result
                facts[key] = value

    return facts, goal

# Load rules from a file.

## Parse a single rule line

In [49]:
def parse_rule_line(line):
    line = line.strip()
    if not line or not line.startswith("IF"):
        return []

    parts = line.split(" THEN ")
    if len(parts) != 2:
        return []

    conditions_str = parts[0].replace("IF ", "").strip()
    result = parts[1].strip()

    # handle OR conditions
    # to make new cond for each one
    if " OR " in conditions_str:
        conditions = [cond.strip() for cond in conditions_str.split(" OR ")]
        return [{"conditions": [cond], "result": result} for cond in conditions]

    # handle AND conditions
    elif " AND " in conditions_str:
        conditions = [cond.strip() for cond in conditions_str.split(" AND ")]
        return [{"conditions": conditions, "result": result}]

    # handle single condition
    else:
        return [{"conditions": [conditions_str], "result": result}]

In [50]:
def load_rules(filename):
    rules = []
    with open(filename, 'r') as file:
        for line in file:
            parsed_rules = parse_rule_line(line)
            rules.extend(parsed_rules)
    return rules


## handle math operators (>, <, =)

In [51]:
def evaluate_condition(condition, facts_dict):

    if '>' in condition:
        key, value = condition.split('>')
        key = key.strip()
        value = float(value.strip())
        return key in facts_dict and isinstance(facts_dict[key], (int, float)) and float(facts_dict[key]) > value
    elif '<' in condition:
        key, value = condition.split('<')
        key = key.strip()
        value = float(value.strip())
        return key in facts_dict and isinstance(facts_dict[key], (int, float)) and float(facts_dict[key]) < value
    elif '=' in condition:
        key, value = condition.split('=')
        key = key.strip()
        value = float(value.strip())
        return key in facts_dict and isinstance(facts_dict[key], (int, float)) and float(facts_dict[key]) == value
    elif ' is ' in condition:
        key, value = condition.split(' is ')
        key = key.strip()
        value = value.strip()
        return key in facts_dict and str(facts_dict[key]) == str(value)
    else:
        return condition in facts_dict and facts_dict[condition] is True

## return the rule in a readable format (if -then )

In [52]:

def format_rule(rule):
    return f"IF {' AND '.join(rule['conditions'])} THEN {rule['result']}"


# Backward chaining

In [53]:
def backward_chain(goal, rules, facts_dict, proved_goals=None, depth=0):

    if proved_goals is None:
        proved_goals = set()

    indent = "  " * depth
    print(f"{indent}Step {depth + 1}: Trying to prove: {goal}")

    #  directly available in facts
    if evaluate_condition(goal, facts_dict):
        print(f"{indent}Step {depth + 1}: Evaluated and true: {goal}")
        proved_goals.add(goal)
        return True

    # goal already proven
    if goal in proved_goals:
        print(f"{indent}Step {depth + 1}: Already known: {goal}")
        return True

    # rules that can prove this goal
    applicable_rules = [rule for rule in rules if rule["result"] == goal]
    if not applicable_rules:
        print(f"{indent}Step {depth + 1}: No applicable rules to prove: {goal}")
        return False


    for rule in applicable_rules:
        print(f"{indent}Step {depth + 1}: Considering rule: {format_rule(rule)}")

        all_conditions_proven = True #initially true
        for idx, condition in enumerate(rule["conditions"]):
            print(f"{indent}  Step {depth + 2}.{idx + 1}: Trying to prove condition: {condition}")
            if not backward_chain(condition, rules, facts_dict, proved_goals, depth + 2):
                all_conditions_proven = False
                break

        if all_conditions_proven:
            print(f"{indent}Step {depth + 1}: Proved: {goal}")
            proved_goals.add(goal)
            return True

    print(f"{indent}Step {depth + 1}: Failed to prove: {goal}")
    return False


# forward chaining

In [54]:

def forward_chaining(rules, initial_facts):

    facts = initial_facts.copy()
    inferred_facts = {}

    iteration = 0
    print_facts_state(iteration, facts)

    new_fact_found = True
    while new_fact_found:
        iteration += 1
        new_fact_found = False
        print(f"\nForward Chaining - Iteration {iteration}:")


        for rule_idx, rule in enumerate(rules):
            all_conditions_met = all(evaluate_condition(condition, facts) for condition in rule["conditions"])

            if all_conditions_met:
                result = rule["result"]


                if ' is ' in result:
                    attr, value = result.split(' is ')
                    attr = attr.strip()
                    value = value.strip()

                    if attr not in facts or str(facts[attr]) != str(value):
                        print(f"  Applied Rule: {format_rule(rule)} -> {attr} is {value}")
                        facts[attr] = value
                        inferred_facts[attr] = value
                        new_fact_found = True
                else:
                    if result not in facts:
                        print(f"  Applied Rule: {format_rule(rule)} -> {result} is True")
                        facts[result] = True
                        inferred_facts[result] = True
                        new_fact_found = True

        print_facts_state(iteration, facts)

    print(f"\nForward chaining completed after {iteration} iterations.")
    return inferred_facts

# debugging facts

In [55]:
def print_facts_state(iteration, facts):
    print(f"\nFacts after iteration {iteration}:")
    for attr, value in sorted(facts.items()):
        print(f"  {attr}: {value}")

In [56]:


def print_all_facts(initial_facts, inferred_facts):

    print("\nAll Known Facts:")
    print("Initial Facts:")
    for attr, value in sorted(initial_facts.items()):
        print(f"  {attr}: {value}")
    print("Inferred Facts:")
    for attr, value in sorted(inferred_facts.items()):
        print(f"  {attr}: {value}")

# read facts & rules from files

In [57]:

facts_dict, goal = load_facts("facts.txt")
rules = load_rules("rules.txt")




In [58]:
print("\nFacts loaded:")
for k, v in facts_dict.items():
    print(f"  {k}: {v}")


Facts loaded:
  seeds: 0.0
  diameter: 7.0
  skin_smell: True
  color: orange


In [59]:
print("\nRules loaded:")
for idx, r in enumerate(rules):
    print(f"  Rule {idx+1}: {format_rule(r)}")


Rules loaded:
  Rule 1: IF shape is long AND color is yellow THEN fruit is banana
  Rule 2: IF shape is round AND color is red AND size is medium THEN fruit is apple
  Rule 3: IF shape is round AND color is red AND size is small THEN fruit is cherry
  Rule 4: IF skin_smell THEN perfumed
  Rule 5: IF fruit is lemon THEN citrus_fruit
  Rule 6: IF fruit is orange THEN citrus_fruit
  Rule 7: IF fruit is pomelo THEN citrus_fruit
  Rule 8: IF fruit is grapefruit THEN citrus_fruit
  Rule 9: IF size is medium AND color is yellow AND perfumed THEN fruit is lemon
  Rule 10: IF size is medium AND color is green THEN fruit is kiwi
  Rule 11: IF size is big AND perfumed AND color is orange AND citrus_fruit THEN fruit is grapefruit
  Rule 12: IF perfumed AND color is orange AND size is medium THEN fruit is orange
  Rule 13: IF perfumed AND color is red AND size is small AND seeds = 0 THEN fruit is strawberry
  Rule 14: IF diameter < 2 THEN size is small
  Rule 15: IF diameter > 10 THEN size is big


# Test

##  test backward chaining

In [60]:
if goal:
    print(f"\nTrying to prove goal: {goal}")
    result = backward_chain(goal, rules, facts_dict)
    print(f"\nCan we prove '{goal}'? {'Yes' if result else 'No'}")


Trying to prove goal: citrus_fruit
Step 1: Trying to prove: citrus_fruit
Step 1: Considering rule: IF fruit is lemon THEN citrus_fruit
  Step 2.1: Trying to prove condition: fruit is lemon
    Step 3: Trying to prove: fruit is lemon
    Step 3: Considering rule: IF size is medium AND color is yellow AND perfumed THEN fruit is lemon
      Step 4.1: Trying to prove condition: size is medium
        Step 5: Trying to prove: size is medium
        Step 5: Considering rule: IF diameter > 2 AND diameter < 10 THEN size is medium
          Step 6.1: Trying to prove condition: diameter > 2
            Step 7: Trying to prove: diameter > 2
            Step 7: Evaluated and true: diameter > 2
          Step 6.2: Trying to prove condition: diameter < 10
            Step 7: Trying to prove: diameter < 10
            Step 7: Evaluated and true: diameter < 10
        Step 5: Proved: size is medium
      Step 4.2: Trying to prove condition: color is yellow
        Step 5: Trying to prove: color is ye

## test forward chaining

In [61]:
inferred_facts = forward_chaining(rules, facts_dict)



Facts after iteration 0:
  color: orange
  diameter: 7.0
  seeds: 0.0
  skin_smell: True

Forward Chaining - Iteration 1:
  Applied Rule: IF skin_smell THEN perfumed -> perfumed is True
  Applied Rule: IF diameter > 2 AND diameter < 10 THEN size is medium -> size is medium

Facts after iteration 1:
  color: orange
  diameter: 7.0
  perfumed: True
  seeds: 0.0
  size: medium
  skin_smell: True

Forward Chaining - Iteration 2:
  Applied Rule: IF perfumed AND color is orange AND size is medium THEN fruit is orange -> fruit is orange

Facts after iteration 2:
  color: orange
  diameter: 7.0
  fruit: orange
  perfumed: True
  seeds: 0.0
  size: medium
  skin_smell: True

Forward Chaining - Iteration 3:
  Applied Rule: IF fruit is orange THEN citrus_fruit -> citrus_fruit is True

Facts after iteration 3:
  citrus_fruit: True
  color: orange
  diameter: 7.0
  fruit: orange
  perfumed: True
  seeds: 0.0
  size: medium
  skin_smell: True

Forward Chaining - Iteration 4:

Facts after iteration 

In [62]:
print_all_facts(facts_dict, inferred_facts)


All Known Facts:
Initial Facts:
  color: orange
  diameter: 7.0
  seeds: 0.0
  skin_smell: True
Inferred Facts:
  citrus_fruit: True
  fruit: orange
  perfumed: True
  size: medium
